In [1]:
import os
import time
import pickle
import keyboard
import numpy as np
import random

import tminterface as tmi
from tminterface.interface import TMInterface, Client

# Useful functions

In [2]:
def discrete_to_continuous(n):

    """
    Equivalents:
    
    0: no action
    1: left
    2: left + acceleration
    3: acceleration
    4: right + acceleration
    5: right
    """

    current_action = {
    'sim_clear_buffer': True,  
    "steer":           0,
    "accelerate":      False, 
    "brake" :          False
    }
    
    if n == 1:
        current_action["steer"] = -65536
    if n == 2:
        current_action["steer"] = -65536
        current_action["accelerate"] = True
    if n == 3:
        current_action["accelerate"] = True
    if n == 4:
        current_action["steer"] = 65536
        current_action["accelerate"] = True
    if n == 5:
        current_action["steer"] = 65536
        
    return current_action
    

def reset_detection(_time, position):
    done = False
    if position[1] < 9.2:
        done = True

    if _time >= 500:
        if velocity()[2] < 1:
            done = True

    return done

# Abstract Client

In [3]:
class AbstractClient(Client):

    def __init__(self):
        super().__init__()
        self.dna = None
        self.period = 1.0
        self.period_ms = np.floor(1000*self.period)
        self.final_state = None
        self.start_state = None
        self.is_finish = False
        self.finish_dna = None
        
    def on_registered(self, iface: TMInterface) -> None:
        iface.execute_command("press delete")
        print(f'Registered to {iface.server_name}')

    def on_run_step(self, iface, _time: int):
        self.action(iface, _time)

    def on_checkpoint_count_changed(self, iface, current: int, target: int):
        if current >= 1 and current == target:
            self.is_finish = True
            self.finish_dna = self.dna.copy()

    def reset_detection(self, _time, state):
        
        if state.position[1] < 9.2:
            return True
    
        if _time >= 500:
            local_velocity = state.scene_mobil.current_local_speed
            local_velocity = np.array(list(local_velocity.to_numpy()))
            local_velocity = local_velocity*3.6 
            if local_velocity[2] < 1:
                return True

        if state.scene_mobil.has_any_lateral_contact:
            return True
    
        return False

    def action(self, iface, _time: int):
        if _time >= 0 and _time < len(self.dna)*self.period_ms:
            
            action = self.dna[int(np.floor(_time/self.period_ms))]
            command = discrete_to_continuous(action)
            iface.set_input_state(**command)
            
            if self.reset_detection(_time, iface.get_simulation_state()):
                self.finish(iface, _time)
                
        if _time == len(self.dna)*self.period_ms:
            self.finish(iface, _time)

    def finish(self, iface,  _time):
        self.final_state = iface.get_simulation_state()
        iface.rewind_to_state(self.start_state)

# Training Client and Replay Client

In [4]:
class TrainingClient(AbstractClient):

    def __init__(self, period=0.5, n_trials=200):
        super().__init__()
        # DNA = memory + gene
        self.memory = np.array([])
        self.past_tries = [[]]
        self.gene = None
        self.generate_dna()
        self.period = period
        self.period_ms = np.floor(1000*self.period)
        self.max_trials = n_trials
        self.n_trial = 1

        self.best_gene = self.gene
        self.best_perf = 0
        self.anchor = 0

    def generate_dna(self):
        if self.gene is not None:
            self.past_tries.append(list(self.gene))
        self.gene = np.random.randint(low=1, high=6, size=5)
        while list(self.gene) in self.past_tries:
            self.gene = np.random.randint(low=1, high=6, size=5)
        self.dna = np.concatenate([self.memory, self.gene])

    def on_run_step(self, iface, _time: int):
        self.action(iface, _time)
        if _time == self.anchor:
            self.start_state = iface.get_simulation_state()
            
    def finish(self, iface, _time):
        self.n_trial += 1
        self.final_state = iface.get_simulation_state()

        # OBECTIVE FUNCTION CONDITION
        if self.final_state.position[0] > self.best_perf:
            self.best_gene = self.gene
            self.best_perf = self.final_state.position[0]

        if self.n_trial == self.max_trials:
            self.memory = np.concatenate([self.memory, self.best_gene[:1]])
            self.n_trial = 0
            print(self.memory)
            self.past_tries = [[]]
            self.anchor += self.period_ms
            self.best_perf = 0
        
        iface.rewind_to_state(self.start_state)
        self.generate_dna()
        
class ReplayClient(AbstractClient):

    def __init__(self, period, dna): 
        super().__init__()
        self.period = period
        self.period_ms = np.floor(1000*self.period)
        self.dna = dna

    def on_run_step(self, iface, _time: int):
        self.action(iface, _time)
        if _time == 0:
            self.start_state = iface.get_simulation_state()
        
    def finish(self, iface, _time):
        self.final_state = iface.get_simulation_state()
        print(self.final_state.position)
        iface.rewind_to_state(self.start_state)

## Training Client

In [5]:
interface = TMInterface()
client = TrainingClient(period= 0.5, n_trials=75)

interface.register(client)
print("Start")

while client.is_finish is False:
    time.sleep(0.001)

    if keyboard.is_pressed("q"):
        print("Keybord Interrupt")
        break

if client.is_finish:
    best_memory = client.memory + client.finish_dna
else:
    best_memory = client.memory

interface.close()
best_dna = client.memory
print(best_dna)

Start
Registered to TMInterface0
[2.]
[2. 3.]
[2. 3. 4.]
[2. 3. 4. 2.]
[2. 3. 4. 2. 1.]
[2. 3. 4. 2. 1. 4.]
[2. 3. 4. 2. 1. 4. 3.]
[2. 3. 4. 2. 1. 4. 3. 3.]
[2. 3. 4. 2. 1. 4. 3. 3. 2.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2. 5.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2. 5. 5.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2. 5. 5. 3.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2. 5. 5. 3. 4.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2. 5. 5. 3. 4. 2.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2. 5. 5. 3. 4. 2. 2.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2. 5. 5. 3. 4. 2. 2. 2.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2. 5. 5. 3. 4. 2. 2. 2. 3.]
[2. 3. 4. 2. 1. 4. 3. 3. 2. 2. 4. 4. 4. 4. 2. 5. 5. 3. 4. 2. 2. 2. 3. 5.]
[2. 3. 4. 2. 1. 4. 

## Replay Client

In [6]:
interface = TMInterface()
client = ReplayClient(period= 0.5, dna=best_memory)

interface.register(client)
print("Start")

while True:
    time.sleep(0.001)

    if keyboard.is_pressed("q"):
        print("Keybord Interrupt")
        break

interface.close()

Start
Registered to TMInterface0
[347.8049011230469, 9.383550643920898, 467.6280212402344]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
[329.71484375, 9.384645462036133, 474.2113342285156]
Keybord Interrupt


# Testing

In [75]:
best_memory = np.array([2., 3., 4., 3., 2., 2., 4., 2., 3., 3., 3., 5., 4., 4., 3., 4., 2.,
       5., 5., 2., 1., 2., 2., 2., 3., 3., 4., 4., 5., 4., 3., 4., 1., 1.,
       2., 5., 3., 1., 2., 4., 2., 2., 3., 4.])

In [74]:
client.memory

array([2., 3., 4., 3., 2., 2., 4., 2., 3., 3., 3., 5., 4., 4., 3., 4., 2.,
       5., 5., 2., 1., 2., 2., 2., 3., 3., 4., 4., 5., 4., 3., 4., 1., 1.,
       2., 5., 3., 1., 2., 4., 2., 2., 3., 4.])